In [1]:
!pip install openprompt
!wget -q --content-disposition https://cloud.tsinghua.edu.cn/f/cd464aed35fc49429971/?dl=1
!tar -zxvf webnlg_2017.tar.gz
!rm -rf webnlg_2017.tar.gz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.4/146.4 kB 1.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 12.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 36.5 MB/s eta 0:00:00
  Attempting uninstall: sentencepiece
    Found existing installation: sentencepiece 0.1.97
    Uninstalling sentencepiece-0.1.97:
      Successfully uninstalled sentencepiece-0.1.97
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-io 0.21.0 requires tensorflow-io-gcs-filesystem==0.21.0, which is not installed.
beatrix-jupyterlab 3.1.7 requires google-cloud-bigquery-storage, which is not installed.
tfx-bsl 1.9.0 requires google-api-python

In [2]:
import torch
from openprompt.data_utils.conditional_generation_dataset import WebNLGProcessor
dataset = {}
dataset_path = "./webnlg_2017"
dataset['train'] = WebNLGProcessor().get_train_examples(dataset_path)
dataset['validation'] = WebNLGProcessor().get_dev_examples(dataset_path)
dataset['test'] = WebNLGProcessor().get_test_examples(dataset_path)


In [3]:
from openprompt.plms import load_plm
plm, tokenizer, model_config, WrapperClass = load_plm("gpt2", "gpt2")

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/523M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Using pad_token, but it is not set yet.


In [4]:
from openprompt.prompts.prefix_tuning_template import PrefixTuningTemplate
mytemplate = PrefixTuningTemplate(model=plm,  tokenizer=tokenizer, text=' {"placeholder":"text_a"} {"special": "<eos>"} {"mask"} ', using_decoder_past_key_values=False)


In [20]:
dataset['train'][111]

{
  "guid": "111",
  "label": null,
  "meta": {},
  "text_a": " | Afonso_Pena_International_Airport : runwayLength : 2215.0",
  "text_b": "",
  "tgt_text": "Afonso Pena International Airport has a runway length of 2215.0."
}

In [5]:
wrapped_example = mytemplate.wrap_one_example(dataset['train'][0])
print(wrapped_example)


[[{'text': '  | Aarhus_Airport : cityServed : "Aarhus, Denmark"', 'loss_ids': 0, 'shortenable_ids': 1}, {'text': '<eos>', 'loss_ids': 0, 'shortenable_ids': 0}, {'text': '<mask>', 'loss_ids': 1, 'shortenable_ids': 0}], {'guid': '0', 'tgt_text': 'The Aarhus is the airport of Aarhus, Denmark.'}]


In [6]:
from openprompt import PromptDataLoader
train_dataloader = PromptDataLoader(dataset=dataset["train"], template=mytemplate, tokenizer=tokenizer,
    tokenizer_wrapper_class=WrapperClass, max_seq_length=256, decoder_max_length=256,
    batch_size=5,shuffle=True, teacher_forcing=True, predict_eos_token=True, # be sure to pass predict_eos_token=True if your template doesn't contain one, or you model may fail to stop generation.
    truncate_method="head")

validation_dataloader = PromptDataLoader(dataset=dataset["validation"], template=mytemplate, tokenizer=tokenizer,
    tokenizer_wrapper_class=WrapperClass, max_seq_length=256, decoder_max_length=256,
    batch_size=5,shuffle=False, teacher_forcing=False, predict_eos_token=True,
    truncate_method="head")

test_dataloader = PromptDataLoader(dataset=dataset["test"], template=mytemplate, tokenizer=tokenizer,
    tokenizer_wrapper_class=WrapperClass, max_seq_length=256, decoder_max_length=256,
    batch_size=5,shuffle=False, teacher_forcing=False, predict_eos_token=True,
    truncate_method="head")

tokenizing: 18025it [00:19, 910.29it/s]
tokenizing: 872it [00:00, 1562.36it/s]
tokenizing: 1862it [00:01, 1501.03it/s]


In [7]:
import openprompt
from openprompt import PromptForGeneration


use_cuda = True
prompt_model = PromptForGeneration(plm=plm,template=mytemplate, freeze_plm=True,tokenizer=tokenizer, plm_eval_mode=False)
if use_cuda:
    prompt_model=  prompt_model.cuda()

In [8]:
from transformers import AdamW


no_decay = ["bias", "LayerNorm.weight"]
optimizer_grouped_parameters = [
{
    "params": [p for n, p in mytemplate.named_parameters() if (not any(nd in n for nd in no_decay)) and p.requires_grad],
    "weight_decay": 0.0,
},
{
    "params": [p for n, p in mytemplate.named_parameters() if any(nd in n for nd in no_decay) and p.requires_grad],
    "weight_decay": 0.0,
},
]


In [9]:
optimizer = AdamW(optimizer_grouped_parameters, lr=1e-5, eps=1e-8)

from transformers.optimization import get_linear_schedule_with_warmup

tot_step  = len(train_dataloader)*5
scheduler = get_linear_schedule_with_warmup(optimizer, 0, tot_step)

/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [10]:
from openprompt.utils.metrics import generation_metric

def evaluate(prompt_model, dataloader):
    generated_sentence = []
    groundtruth_sentence = []
    prompt_model.eval()

    for step, inputs in enumerate(dataloader):
        if use_cuda:
            inputs = inputs.cuda()
        _, output_sentence = prompt_model.generate(inputs, **generation_arguments)
        generated_sentence.extend(output_sentence)
        groundtruth_sentence.extend(inputs['tgt_text'])
    score = generation_metric(generated_sentence, groundtruth_sentence, "sentence_bleu")
    print("test_score", score, flush=True)
    return generated_sentence


In [11]:
generation_arguments = {
    "max_length": 512,
    "max_new_tokens": None,
    "min_length": 5,
    "temperature": 1.0,
    "do_sample": False,
    "top_k": 0,
    "top_p": 0.9,
    "repetition_penalty": 1.0,
    "num_beams": 5,
    "bad_words_ids": [[628], [198]]
}


In [12]:
global_step = 0
tot_loss = 0
log_loss = 0
for epoch in range(2):
    prompt_model.train()
    for step, inputs in enumerate(train_dataloader):
        global_step +=1
        if use_cuda:
            inputs = inputs.cuda()
        loss = prompt_model(inputs)
        loss.backward()
        tot_loss += loss.item()
        torch.nn.utils.clip_grad_norm_(mytemplate.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
        if global_step %500 ==0:
            print("Epoch {}, global_step {} average loss: {} lr: {}".format(epoch, global_step, (tot_loss-log_loss)/500, scheduler.get_last_lr()[0]), flush=True)
            log_loss = tot_loss

Epoch 0, global_step 500 average loss: 147.2527066307068 lr: 9.722607489597781e-06
Epoch 0, global_step 1000 average loss: 56.99748106384278 lr: 9.445214979195563e-06
Epoch 0, global_step 1500 average loss: 49.78494994354248 lr: 9.167822468793343e-06
Epoch 0, global_step 2000 average loss: 47.42565599632263 lr: 8.890429958391125e-06
Epoch 0, global_step 2500 average loss: 46.2406116065979 lr: 8.613037447988904e-06
Epoch 0, global_step 3000 average loss: 43.77057800674439 lr: 8.335644937586686e-06
Epoch 0, global_step 3500 average loss: 44.046577253341674 lr: 8.058252427184466e-06
Epoch 1, global_step 4000 average loss: 41.6276708278656 lr: 7.780859916782248e-06
Epoch 1, global_step 4500 average loss: 42.05457205200195 lr: 7.503467406380028e-06
Epoch 1, global_step 5000 average loss: 41.36603812599182 lr: 7.226074895977809e-06
Epoch 1, global_step 5500 average loss: 41.35049138069153 lr: 6.94868238557559e-06
Epoch 1, global_step 6000 average loss: 40.482076379776004 lr: 6.67128987517337

In [13]:
generated_sentence = evaluate(prompt_model, test_dataloader)

with open(f"./Generated_sentence_webnlg_gpt2.txt",'w') as f:
    for i in generated_sentence:
        f.write(i+"\n")

test_score 0.35283928333446424


In [25]:
wrapped_example = mytemplate.wrap_one_example(dataset['train'][0])
wrapped_example

[[{'text': '  | Aarhus_Airport : cityServed : "Aarhus, Denmark"',
   'loss_ids': 0,
   'shortenable_ids': 1},
  {'text': '<eos>', 'loss_ids': 0, 'shortenable_ids': 0},
  {'text': '<mask>', 'loss_ids': 1, 'shortenable_ids': 0}],
 {'guid': '0', 'tgt_text': 'The Aarhus is the airport of Aarhus, Denmark.'}]

In [14]:
from openprompt.data_utils import InputExample

small_dataset = [
    
        InputExample(
        
            guid = 0,
            text_a = " | Aarhus_Airport : cityServed : \"Aarhus, Denmark\"",
            tgt_text = "The Aarhus is the airport of Aarhus, Denmark."
        ),
]

In [15]:
dl = PromptDataLoader(dataset=small_dataset, template=mytemplate, tokenizer=tokenizer,
    tokenizer_wrapper_class=WrapperClass, max_seq_length=256, decoder_max_length=256,
    batch_size=5,shuffle=True, teacher_forcing=True, predict_eos_token=True, # be sure to pass predict_eos_token=True if your template doesn't contain one, or you model may fail to stop generation.
    truncate_method="head")

tokenizing: 1it [00:00, 511.94it/s]


In [16]:
for ip in dl:
    print(prompt_model.generate(ip.cuda(), **generation_arguments)[1])



['Aarhus Airport serves Aarhus, Denmark.']
